In [18]:
import pathlib
import tensorflow as tf
from PIL import Image
import numpy as np

from model.resnet import ResNet2

In [8]:
data_dir = pathlib.Path('data_generation/data')
print(data_dir)
print(len(list(data_dir.glob('Isothermal/*.png'))))
sim0s = list(data_dir.glob('Isothermal/*.png'))
# Image.open(str(sim0s[0]))

data_generation\data
98


In [14]:
BATCH_SIZE = 32
IMG_HEIGHT = 512
IMG_WIDTH = 512

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 123,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 123,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE)
class_names = train_ds.class_names
print(class_names)

Found 1326 files belonging to 8 classes.
Using 1061 files for training.
Found 1326 files belonging to 8 classes.
Using 265 files for validation.
['Chameleon', 'DeVaucoulers', 'Exponential', 'Gaussian', 'Isothermal', 'Massless', 'Sersic', 'gNFW']


In [15]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 512, 512, 3)
(32,)


In [16]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [17]:
num_classes = 8

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# model = ResNet2(shape=(512,512,3), classes=10)

# model = model.model

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  normalized_train_ds,
  validation_data=normalized_val_ds,
  epochs=3
)

Epoch 1/3
34/34 [==============================] - 98s 3s/step - loss: 2.0954 - accuracy: 0.1310 - val_loss: 2.0644 - val_accuracy: 0.1660
Epoch 2/3
34/34 [==============================] - 96s 3s/step - loss: 2.0442 - accuracy: 0.1970 - val_loss: 2.1218 - val_accuracy: 0.1736
Epoch 3/3
34/34 [==============================] - 88s 3s/step - loss: 1.8281 - accuracy: 0.3553 - val_loss: 2.2536 - val_accuracy: 0.1736


In [19]:
model2 = ResNet2(shape=(512,512,3), classes=10).model

model2.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
34/34 [==============================] - 509s 15s/step - loss: 46.8550 - accuracy: 0.1131 - val_loss: 542714.1875 - val_accuracy: 0.1396
Epoch 2/3
34/34 [==============================] - 504s 15s/step - loss: 4.7702 - accuracy: 0.1225 - val_loss: 541.0684 - val_accuracy: 0.0981
Epoch 3/3
34/34 [==============================] - 504s 15s/step - loss: 2.7786 - accuracy: 0.1395 - val_loss: 157.0589 - val_accuracy: 0.0792


In [20]:
num_classes = 8

model3 = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# model = ResNet2(shape=(512,512,3), classes=10)

# model = model.model

model3.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model3.fit(
  normalized_train_ds,
  validation_data=normalized_val_ds,
  epochs=3
)

Epoch 1/3
34/34 [==============================] - 100s 3s/step - loss: 2.7262 - accuracy: 0.1433 - val_loss: 2.0788 - val_accuracy: 0.1585
Epoch 2/3
34/34 [==============================] - 93s 3s/step - loss: 2.0734 - accuracy: 0.1960 - val_loss: 2.0928 - val_accuracy: 0.1472
Epoch 3/3
34/34 [==============================] - 85s 2s/step - loss: 2.0114 - accuracy: 0.3393 - val_loss: 2.1847 - val_accuracy: 0.1660


In [21]:
num_classes = 8

model4 = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  
  tf.keras.layers.Dense(num_classes)
])

# model = ResNet2(shape=(512,512,3), classes=10)

# model = model.model

model4.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model4.fit(
  normalized_train_ds,
  validation_data=normalized_val_ds,
  epochs=3
)

Epoch 1/3
34/34 [==============================] - 94s 3s/step - loss: 2.0858 - accuracy: 0.1574 - val_loss: 2.0791 - val_accuracy: 0.1208
Epoch 2/3
34/34 [==============================] - 88s 3s/step - loss: 2.0529 - accuracy: 0.2074 - val_loss: 2.1653 - val_accuracy: 0.1774
Epoch 3/3
34/34 [==============================] - 86s 3s/step - loss: 1.9484 - accuracy: 0.2828 - val_loss: 2.1029 - val_accuracy: 0.1736
